In [1]:
import pandas as pd
import openpyxl as exc
from openpyxl import workbook as wb
from openpyxl import load_workbook
import xlrd
import pyexcel as p
import os

In [2]:
# og_path = r"C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\materials\\xl_97"
og_path = r"C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\"
end_path = r"C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_10\\"
export = r"C:\Users\willd\OneDrive\Desktop\Belair\materials\export\price_data.xlsx"
matches = ['precast', 'manholes']

In [3]:
#Convert xl 97 format to xl 10 
def convert_97_to_10(og, end):
    for item in os.listdir(og):
        full_path = os.path.join(og, item)
        print(full_path)
        name = item.split('.')[0]
        new_name = name + '.xlsx'
        print(new_name)
        p.save_book_as(file_name=(full_path),dest_file_name = end + new_name)

# returns job and address from df 
def job_local(df):
    job = df.iloc[4,3]
    locat = df.iloc[8, 3]
    return job, locat
    

# function to get material vendors and use as column names 
def get_vendors(df):
    ven_df_list = []
    ref1 = df.index[df['Product'] == 'Quote'].tolist()
    ref2 = df.index[df['Product'] == 'Material Prices'].tolist()
    for ref in range(len(ref1)):
        ven_dict = {'Q1':'Q1', 'Q2':'Q2', 'Q3':'Q3', 'Q4':'Q4'}
        number_of_venders = ref2[ref] - ref1[ref]
        dict_ind = 0
        ref3 = ref1[ref] + 1
        for vend in range(number_of_venders-1):
            q = 'Q' + str(dict_ind + 1)
            ven_dict[q] = df['Sub'].iloc[ref3]
            dict_ind += 1
            ref3 += 1
        ven_df = pd.DataFrame.from_dict(ven_dict, orient='index')
        ven_df.columns = ['vendors']
        ven_df['Scope'] = df['Scope'][ref3]
        ven_df.index.name='variable'
        ven_df_list.append(ven_df)
        

            
    ven_df_list = pd.concat(ven_df_list)
    return ven_df_list


In [4]:
#covert
convert_97_to_10(og_path, end_path)

C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\Vendor RFQ Analysis 01.xls
Vendor RFQ Analysis 01.xlsx
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\Vendor RFQ Analysis 02.xls
Vendor RFQ Analysis 02.xlsx
WARNING *** file size (51622) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\Vendor RFQ Analysis 03.xls
Vendor RFQ Analysis 03.xlsx
WARNING *** file size (27095) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_97\\Vendor RFQ Analysis 04 .xls
Vendor RFQ Analysis 04 .xlsx
WARNING *** file size (75758) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\willd\OneDrive\Desktop\Belair\materials\xl_

In [9]:
full_melt = []

for item in os.listdir(end_path):
    full_path = os.path.join(end_path, item)
    xl = pd.read_excel(full_path)
    df = pd.DataFrame(xl)
    job, location = job_local(df)
    # add job and location column 
    df['Job'] = job
    df['Address'] = location
    
    
    # job/location

    #remove nas for first row/ reset index / add date column 
    no_na_prod = df[df['Belair Sitework Services'].notna()]
    no_na_prod.reset_index()
    date = df.iloc[len(df)-1, 0]
    #rename columns
    no_na_prod = no_na_prod.rename(columns={'Belair Sitework Services': 'Product','Unnamed: 1': 'Sub', 'Unnamed: 5': 'Scope',
                                            'Unnamed: 8': 'QTY','Unnamed: 10': 'Unit'
                            , 'Unnamed: 4': 'Date','Unnamed: 13': 'Q1','Unnamed: 16': 'Q2','Unnamed: 19': 'Q3','Unnamed: 22': 'Q4' })
    no_na_prod['Date'] = date
    # remove unnecesary columns 
    reduced = no_na_prod.drop(df.columns[[2,6,7,9,11,12,14,15,17,18,20,21,23]], axis=1)
    reduced = reduced.reset_index()


    # get reference index to locate data 
    reference_row_index = reduced.index[reduced['Product'] == 'Vendor Information'].tolist()

    #create an exted list with final index 
    long_list = reference_row_index.copy()
    long_list.append(len(reduced))

    # fill the scope column with relevent scope 
    for ref_ind in range(len(reference_row_index)):
        scope = reduced.iloc[(reference_row_index[ref_ind]-1), 3]
        scope = scope.replace(' ', '').lower()
        if any(x in scope for x in matches):
            scope = 'precast'
            
        reduced['Scope'][reference_row_index[ref_ind]:long_list[ref_ind+1]] = scope
    # remove more unnecesary columns 
    reduced = reduced.drop(reduced.columns[[0,3]], axis=1)
    reduced
    # drop rows with blank scope field
    no_na_scope = reduced[reduced['Scope'].notna()].reset_index(drop=True)
    # get list of dfs of vendors to replace q1-q4 after melting
    vendors = get_vendors(no_na_scope)
    # remove all rows without prices 
    no_na_price = no_na_scope[no_na_scope['Q1'].notna()]
    #renove characters
    
    move_qs = pd.melt(no_na_price, id_vars=['Product', 'Scope'], value_vars=['Q1', 'Q2', 'Q3', 'Q4'], ignore_index=True)
    move_qs['value'] = move_qs['value'].replace('[^.0-9\-]', '', regex=True)

    lcols, rcols = ['variable','Scope'], ['variable','Scope'] #mappings
    move_qs["variable"] = (move_qs.merge(vendors, left_on=lcols, right_on=rcols, how="left")
                 ["vendors"].combine_first(move_qs["variable"])
            )
    
    full_melt.append(move_qs)
    
full = pd.concat(full_melt)    
    
no_blank_value = full[full['value'] != '']
no_blank_value['value'] = pd.to_numeric(no_blank_value['value'], errors='coerce')
# no_blank_value['value'] = no_blank_value['value']
no_blank_value['value'] = no_blank_value['value'].astype(float)
no_zero = no_blank_value[no_blank_value['value'] > 0]

scope_dict = {'aggregate':[], 'precast':[], 'concrete':[],'rcppipe':[], 'waterworks':[]}
for scp in scope_dict.keys():
    scp_df = no_zero[no_zero['Scope'] == scp]
    scope_dict[scp].append(scp_df)
for key in scope_dict.keys():
    combined = pd.concat(scope_dict[key])
    scope_dict[key] = combined.groupby(['Product', 'variable'])['value'].agg(['min', 'max', 'mean', 'count']).reset_index()
# no vendors option
    #     scope_dict[key] = combined.groupby('Product')['value'].agg(['min', 'max', 'mean', 'count']).reset_index()
    scope_dict[key]['mean'] = round(scope_dict[key]['mean'],2)


C:\Users\willd\AppData\Local\Temp\ipykernel_7092\548854919.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced['Scope'][reference_row_index[ref_ind]:long_list[ref_ind+1]] = scope
C:\Users\willd\AppData\Local\Temp\ipykernel_7092\548854919.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced['Scope'][reference_row_index[ref_ind]:long_list[ref_ind+1]] = scope
C:\Users\willd\AppData\Local\Temp\ipykernel_7092\548854919.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

C:\Users\willd\AppData\Local\Temp\ipykernel_7092\548854919.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced['Scope'][reference_row_index[ref_ind]:long_list[ref_ind+1]] = scope
C:\Users\willd\AppData\Local\Temp\ipykernel_7092\548854919.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced['Scope'][reference_row_index[ref_ind]:long_list[ref_ind+1]] = scope
C:\Users\willd\AppData\Local\Temp\ipykernel_7092\548854919.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [10]:
# save pricing data to xl sheets 
writer = pd.ExcelWriter(export, engine="xlsxwriter")
for key in scope_dict.keys():
  scope_dict[key].to_excel(writer, sheet_name= key.upper())  
writer.close()